In [ ]:
#Getting todays Relevant tweets
!git clone --depth=1 https://github.com/adamlesniak/twint.git -q
!cd /content/twint && pip3 install . -r requirements.txt -q
!pip install vadersentiment -q

  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
     |████████████████████████████████| 1.3 MB 6.6 MB/s 
     |████████████████████████████████| 263 kB 27.0 MB/s 
     |████████████████████████████████| 373 kB 18.0 MB/s 
     |████████████████████████████████| 94 kB 1.8 MB/s 
     |████████████████████████████████| 271 kB 42.2 MB/s 
     |████████████████████████████████| 291 kB 46.7 MB/s 
     |████████████████████████████████| 58 kB 5.0 MB/s 
     |████████████████████████████████| 138 kB 42.6 MB/s 
     |████████████████████████████████| 63 kB 1.7 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages th

In [ ]:

#Imports
import twint
import torch
import time
import datetime
import pandas as pd

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

import nest_asyncio
nest_asyncio.apply()

In [ ]:
#tweeter scraper

#change delta days to go back more days
def scrapetweets(end_time = datetime.datetime.now(), delta_days = 1, delta = 100, query="stock OR Nasdaq"):
  #setting up start and end times
  end_time = end_time-datetime.timedelta(days=delta_days)
  #till now 100 seconds ago
  start_time = end_time-datetime.timedelta(seconds=delta)
  #setting up the configuration for twint
  c = twint.Config()
  c.Lang = "en"
  c.Store_csv = True
  c.Search = query
  c.Since = start_time.strftime("%Y-%m-%d %H:%M:%S")
  c.Until = end_time.strftime("%Y-%m-%d %H:%M:%S")
  #c.Hide_output = True
  #setting up where to put output
  filepath = str(delta_days) + "DaysAgoTweets"+ ".csv"
  c.Output = filepath
  #getting the tweets
  twint.run.Search(c)
  #save them in new by reading the file
  new = pd.read_csv(filepath)
  return new


#function to get sentiment
def get_sentiment(df):
  #getting the positive ones  applying the analyzer on the tweet and getting the three values
  df["pos"] = df.apply(lambda row: analyzer.polarity_scores(row["tweet"])["pos"], axis=1)
  df["compound"] = df.apply(lambda row: analyzer.polarity_scores(row["tweet"])["compound"], axis=1)
  df["neg"] = df.apply(lambda row: analyzer.polarity_scores(row["tweet"])["neg"], axis=1)
  sentval = df["compound"].mean()
  # if we need df["pos"].mean(), df["compound"].mean(),df["neg"].mean()
  print(df["pos"].mean())
  print(df["compound"].mean())
  print(df["neg"].mean())
  return sentval


In [ ]:
sentiment = get_sentiment(scrapetweets(delta_days = 1))

'''
so compound value per tweet 
is between -1 to 1 the mean value is our overall sentiment score
so the overall sentiment can be between -100 to 100
'''

OverallSentiment = sentiment * 100
OverallSentiment = "{:.2f}".format(OverallSentiment)
print("The Sentiment Score for today was",OverallSentiment)